In [50]:
!pip install transformers tensorflow nltk

In [51]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
import tensorflow as tf
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
lem = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:

def process(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text = text.split(" ")
  text = [word for word in text]
  text = [lem.lemmatize(word) for word in text]
  text = ' '.join([word for word in text])
  return text

In [53]:
text = []
labels = []

with open('/content/drive/MyDrive/btp/dataset.csv') as fp:
  reader = csv.reader(fp,delimiter=',')
  for row in reader:
    text.append(process(row[2]))
    labels.append(row[3])
text = text[1:]
labels = labels[1:]
for i,val in enumerate(labels):
  labels[i] = int(labels[i])
tot_size = len(text)
train_ratio = 1
train_len=int(train_ratio*tot_size)
train_text = text[:train_len]
train_labels = labels[:train_len]
# val_text = text[train_len:]
# val_labels = labels[train_len:]




In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import json
training_percentage = 0

data = dict(json.load(open("/content/drive/MyDrive/btp/sarcasm_data.json")))
sentences = []
labels = []

for i,item in data.items():
    sentences.append(process(item['utterance']))
    labels.append((item['sarcasm'])*1)
training_size = int(training_percentage*len(sentences))
# train_text = sentences[:training_size]
# train_labels = labels[:training_size]
val_text = sentences[training_size:]
val_labels = labels[training_size:]

In [56]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [57]:
train_encodings = tokenizer(train_text,truncation=True,padding=True)

val_encodings = tokenizer(val_text,truncation=True,padding=True)

In [58]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [59]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
model.fit(train_dataset.shuffle(len(train_text)).batch(32),
          epochs=10,
          batch_size=32)

Epoch 1/10
835/835 [==============================] - 801s 936ms/step - loss: 0.3162 - accuracy: 0.8631
Epoch 2/10
835/835 [==============================] - 782s 936ms/step - loss: 0.1796 - accuracy: 0.9303
Epoch 3/10
835/835 [==============================] - 782s 937ms/step - loss: 0.1143 - accuracy: 0.9573
Epoch 4/10
835/835 [==============================] - 783s 938ms/step - loss: 0.0810 - accuracy: 0.9711
Epoch 5/10
835/835 [==============================] - 783s 938ms/step - loss: 0.0645 - accuracy: 0.9782
Epoch 6/10
835/835 [==============================] - 783s 937ms/step - loss: 0.0507 - accuracy: 0.9827
Epoch 7/10
835/835 [==============================] - 783s 938ms/step - loss: 0.0433 - accuracy: 0.9843
Epoch 8/10
835/835 [==============================] - 783s 937ms/step - loss: 0.0324 - accuracy: 0.9888
Epoch 9/10
835/835 [==============================] - 780s 935ms/step - loss: 0.0295 - accuracy: 0.9902
Epoch 10/10
835/835 [==============================] - 779s 933m

In [62]:
model.evaluate(val_dataset.shuffle(len(val_text)).batch(16),
               return_dict=True,
               batch_size=16)

44/44 [==============================] - 14s 233ms/step - loss: 2.6696 - accuracy: 0.5159


{'accuracy': 0.5159420371055603, 'loss': 2.6695992946624756}

In [ ]:
model.save_pretrained('/content/drive/MyDrive/BTP/roberta-base')